In [37]:
from linares_plot import *
import os

In [38]:
def circdist(a1,a2):
    ## Returns the minimal distance in angles between to angles 
    op1=abs(a2-a1)
    angs=[a1,a2]
    op2=min(angs)+(360-max(angs))
    options=[op1,op2]
    return min(options)



def decoding_spikes_angles_err(df, size_test=0.1):
    #### Input : dataframe with three columns: (spikes, behaviour and neuron label)
    ###Take off nans
    df = df.loc[df.iloc[:,0]<9999]
    df.columns=['spikes', 'beh', 'neuron']
    neuron_means=[]
    ### Train and test on the same neuron --> append the error of each neuron
    pop_err=[]
    
    for Neur in df.neuron.unique():
        ### same neuron 5 times --> mean value
        neur_err=[]
        for rep in range(0,5):
            spikes_train, spikes_test, beh_train, beh_test = train_test_split(df.groupby('neuron').get_group(Neur)['spikes'],
                                                                              df.groupby('neuron').get_group(Neur)['beh'],
                                                                              test_size=size_test)  

            ######## Trainning #########
            ## X matrix (intercept and spikes)
            X = np.column_stack([np.ones(np.shape(spikes_train)[0]),spikes_train])
            ## Y (sinus and cos)
            sinus =np.sin([np.radians(np.array(beh_train)[i]) for i in range(0, len(beh_train))])
            cosinus = np.cos([np.radians(np.array(beh_train)[i]) for i in range(0, len(beh_train))])
            Y = np.column_stack([cosinus, sinus])
            ### one OLS for sin and cos: output: beta of intercetp and bea of spikes (two B intercepts and 2 B for spikes )
            Y = Y.astype(float) #to make it work in the cluster
            X = X.astype(float)
            model = sm.OLS(Y, X)
            ##train the model
            fit=model.fit()

            ######### Testing ###########
            X = np.column_stack([np.ones(np.shape(spikes_test)[0]),spikes_test])
            p = fit.predict(X)
            x = p[:,0]
            y = p[:,1]
            #####
            ##### Error --> take the resulting vector in sin/cos space
            ### from sin and cos get the angle (-pi, pi)
            #pred_angle = [ np.degrees(np.arctan2(y[i], x[i]) + np.pi) for i in range(0, len(y))]
            pred_angle = [ np.degrees(np.arctan2(y[i], x[i])) for i in range(0, len(y))]
            for i in range(0, len(pred_angle)):
                if pred_angle[i]<0:
                    pred_angle[i]=360+pred_angle[i]
            ##
            #error=[ circdist(beh_test[i], pred_angle[i]) for i in range(0, len(pred_angle))]
            error=[ circdist(beh_test.values[i], pred_angle[i]) for i in range(0, len(pred_angle))]
            
            #low_value --> predicted positionns close to real
            neur_err.append(np.mean(error))
            ####
        
        #####       
        pop_err.append(np.mean(neur_err))
        ####
    
    return pop_err




In [50]:
path_CTD = 'C:\\Users\\David\\Desktop\\IDIBAPS\\Gottlib_data\\CTD'


In [51]:
pfc_100 = pd.read_excel( os.path.join(path_CTD, 'pfc_100.xlsx'))
pfc_200 = pd.read_excel( os.path.join(path_CTD, 'pfc_200.xlsx'))
pfc_300 = pd.read_excel( os.path.join(path_CTD, 'pfc_300.xlsx'))
pfc_900 = pd.read_excel( os.path.join(path_CTD, 'pfc_900.xlsx'))

In [52]:
lip_100 = pd.read_excel( os.path.join(path_CTD, 'lip_100.xlsx'))
lip_200 = pd.read_excel( os.path.join(path_CTD, 'lip_200.xlsx'))
lip_300 = pd.read_excel( os.path.join(path_CTD, 'lip_300.xlsx'))
lip_900 = pd.read_excel( os.path.join(path_CTD, 'lip_900.xlsx'))

In [41]:
df.targetloc.unique()

array([7, 5, 1, 0, 4, 2, 6, 3], dtype=int64)

In [42]:
import statsmodels.api as sm

def circdist(a1,a2):
    ## Returns the minimal distance in angles between to angles 
    op1=abs(a2-a1)
    angs=[a1,a2]
    op2=min(angs)+(360-max(angs))
    options=[op1,op2]
    return min(options)

def cross_decoding_spikes_angles_err_all(df_train, df_test):
    #### Input : dataframe with three columns: (spikes, behaviour and neuron label)
    ###Take off nans
    #df_train = df_train.loc[df_train.iloc[:,0]<9999]
    df_train.columns=['spikes', 'beh', 'neuron']
    #df_test = df_test.loc[df_test.iloc[:,0]<9999]
    df_test.columns=['spikes', 'beh', 'neuron']
    
    ## Train and test on the same neuron --> append the error of each neuron
    population_err=[]
    
    for Neur in df_train.neuron.unique():
        
        ### same neuron 20 times --> mean value        
        spikes_train= df_train.groupby('neuron').get_group(Neur)['spikes'].values
        spikes_test= df_test.groupby('neuron').get_group(Neur)['spikes'].values
        beh_train= df_train.groupby('neuron').get_group(Neur)['beh'].values
        beh_test= df_test.groupby('neuron').get_group(Neur)['beh'].values

        ######## Trainning #########
        ## X matrix (intercept and spikes)
        X = np.column_stack([np.ones(np.shape(spikes_train)[0]),spikes_train])
        ## Y (sinus and cos)
        sinus =np.sin([np.radians(np.array(beh_train)[i]) for i in range(0, len(beh_train))])
        cosinus = np.cos([np.radians(np.array(beh_train)[i]) for i in range(0, len(beh_train))])
        Y = np.column_stack([cosinus, sinus])
        ### one OLS for sin and cos: output: beta of intercetp and bea of spikes (two B intercepts and 2 B for spikes )
        model = sm.OLS(Y, X)
        ##train the model
        fit=model.fit()

        ######### Testing ###########
        X = np.column_stack([np.ones(np.shape(spikes_test)[0]),spikes_test])
        p = fit.predict(X)
        x = p[:,0]
        y = p[:,1]
        #####
        ##### Error --> take the resulting vector in sin/cos space
        ### from sin and cos get the angle (-pi, pi)
        #pred_angle = [ np.degrees(np.arctan2(y[i], x[i]) + np.pi) for i in range(0, len(y))]
        pred_angle = [ np.degrees(np.arctan2(y[i], x[i])) for i in range(0, len(y))]
        for i in range(0, len(pred_angle)):
            if pred_angle[i]<0:
                pred_angle[i]=360+pred_angle[i]
        ##
        #
        #print(beh_test)
        error=[ circdist(beh_test[i], pred_angle[i]) for i in range(0, len(pred_angle))]

        #low_value --> predicted positionns close to real
        population_err.append(np.mean(error))
        ######
    
    return population_err

In [43]:
df.neuron.unique()


array([  1367701,   1367802,   1367903,   1369702,   1369901,   1370002,
         1370202,   1371202,   1371901,   1372003,   1372301,   1372402,
       189168701, 189169502, 189171003, 189171602, 189172801, 189173101,
       189173801, 189174201, 189175201, 189175801, 189176101, 189176501,
       189176901, 189177402, 189177701,   1891783, 189179001, 189179002,
         1891793, 189179702, 189180001, 189181001, 189181502,   1891819,
       189182202, 189182501, 189182901, 189183201, 189183402,   1891841,
       189184601,   1891857, 189186201, 189187301, 189188901, 189189601,
       189191601, 189191602,   1891918, 189192001, 189192201, 189192401,
         1891935, 189194301, 189195301, 189196301, 189196401, 189209101,
       189209503, 189210001, 189210901], dtype=int64)

In [49]:
df1 = df.loc[(df['times']==500) & (df['neuron']==1367701), ['firing', 'target_angle', 'neuron']] ###tiene que ser centered!
df2 = df.loc[(df['times']==600) & (df['neuron']==1367701), ['firing', 'target_angle', 'neuron']] ###tiene que ser centered!
cross_decoding_spikes_angles_err_all(df1, df2)

[60.657331230582926]

In [ ]:
### cambiar la funcion para que sea en cada neurona (asi poder identificar subpoblaciones)
### buscar en internet como hacer eso? PCA???
### idea! tres fits diferentes! compararlos en cada neurona! :)


In [57]:
def decoder_n(df_train, df_test):
    #### Input : dataframe with three columns: (spikes, behaviour and neuron label)
    df_train.columns=['firing', 'beh']
    df_test.columns=['firing', 'beh']
    
    ## Train and test         
    activity_train= df_train.firing.values
    activity_test= df_test.firing.values
    beh_train= df_train.beh.values
    beh_test= df_test.beh.values

    ######## Trainning #########
    ## X matrix (intercept and spikes)
    X = np.column_stack([np.ones(np.shape(activity_train)[0]), activity_train])
    ## Y (sinus and cos)
    sinus =np.sin([np.radians(np.array(beh_train)[i]) for i in range(0, len(beh_train))])
    cosinus = np.cos([np.radians(np.array(beh_train)[i]) for i in range(0, len(beh_train))])
    Y = np.column_stack([cosinus, sinus])
    ### one OLS for sin and cos: output: beta of intercetp and bea of spikes (two B intercepts and 2 B for spikes )
    model = sm.OLS(Y, X)
    ##train the model
    fit=model.fit()

    ######### Testing ###########
    X = np.column_stack([np.ones(np.shape(activity_test)[0]),activity_test])
    p = fit.predict(X)
    x = p[:,0]
    y = p[:,1]
    #####
    ##### Error --> take the resulting vector in sin/cos space
    ### from sin and cos get the angle (-pi, pi)
    #pred_angle = [ np.degrees(np.arctan2(y[i], x[i]) + np.pi) for i in range(0, len(y))]
    pred_angle = [ np.degrees(np.arctan2(y[i], x[i])) for i in range(0, len(y))]
    for i in range(0, len(pred_angle)):
        if pred_angle[i]<0:
            pred_angle[i]=360+pred_angle[i]
    ##
    #
    #print(beh_test)
    error_trial=[ circdist(beh_test[i], pred_angle[i]) for i in range(0, len(pred_angle))]
    mean_error = np.round(np.mean(error_trial),2)

    return mean_error

In [58]:
df1 = df.loc[(df['times']==500) & (df['neuron']==1367701), ['firing', 'target_angle']] ###tiene que ser centered!
df2 = df.loc[(df['times']==600) & (df['neuron']==1367701), ['firing', 'target_angle']] ###tiene que ser centered!
decoder_n(df1, df2)

60.66

In [63]:
df['times_centered'] = df['times'] - df['fixationtime']

In [64]:
df.loc[df['neuron']==1367701, ['firing', 'target_angle', 'times_centered']]

,firing,target_angle,times_centered
0,-3.637979e-19,225,-300
1,1.382432e-17,225,-200
2,1.127773e-17,225,-100
3,2.691904e-03,225,0
4,1.751611e-02,225,100
...,...,...,...
2242,1.003299e-02,315,800
2243,2.412903e-02,315,900
2244,1.087097e-02,315,1000
2245,3.434188e-06,315,1100


In [67]:
dfn = df.loc[df['neuron']==1367701]
dfn.head()

,Unnamed: 0,firing,times,trial,TDOA,TDconf,targetloc,distLoc,fixationtime,failed,saccadeChoice,monkey,neuron,brain_region,rf_loc,target_angle,times_centered
0,0,-3.637979e-19,0,2,0,3,7,0,300,1,7,Mojo,1367701,DLPFC,270,225,-300
1,1,1.382432e-17,100,2,0,3,7,0,300,1,7,Mojo,1367701,DLPFC,270,225,-200
2,2,1.127773e-17,200,2,0,3,7,0,300,1,7,Mojo,1367701,DLPFC,270,225,-100
3,3,2.691904e-03,300,2,0,3,7,0,300,1,7,Mojo,1367701,DLPFC,270,225,0
4,4,1.751611e-02,400,2,0,3,7,0,300,1,7,Mojo,1367701,DLPFC,270,225,100


In [ ]:
df.loc[df['neuron']==1367701

In [ ]:
    Reconstructions = Parallel(n_jobs = numcores)(delayed(Representation)(signal, testing_angles, WM, WM_t, ref_angle=180, plot=False, intercept=Inter)  for signal in signal_paralel)    #### reconstruction standard (paralel)
